<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span><ul class="toc-item"><li><span><a href="#Основная-информация-о-данных" data-toc-modified-id="Основная-информация-о-данных-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Основная информация о данных</a></span></li><li><span><a href="#Удаление-лишних-данных" data-toc-modified-id="Удаление-лишних-данных-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Удаление лишних данных</a></span></li><li><span><a href="#Расчет-процента-токсичных-твитов" data-toc-modified-id="Расчет-процента-токсичных-твитов-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Расчет процента токсичных твитов</a></span></li><li><span><a href="#Очистка,-лемматизация-текста" data-toc-modified-id="Очистка,-лемматизация-текста-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Очистка, лемматизация текста</a></span></li><li><span><a href="#Разделение-на-выборки" data-toc-modified-id="Разделение-на-выборки-1.5"><span class="toc-item-num">1.5&nbsp;&nbsp;</span>Разделение на выборки</a></span></li><li><span><a href="#Балансировка-классов" data-toc-modified-id="Балансировка-классов-1.6"><span class="toc-item-num">1.6&nbsp;&nbsp;</span>Балансировка классов</a></span></li><li><span><a href="#TF-IDF" data-toc-modified-id="TF-IDF-1.7"><span class="toc-item-num">1.7&nbsp;&nbsp;</span>TF-IDF</a></span></li><li><span><a href="#Промежуточый-вывод" data-toc-modified-id="Промежуточый-вывод-1.8"><span class="toc-item-num">1.8&nbsp;&nbsp;</span>Промежуточый вывод</a></span></li></ul></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span><ul class="toc-item"><li><span><a href="#Логистическая-регрессия" data-toc-modified-id="Логистическая-регрессия-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Логистическая регрессия</a></span></li><li><span><a href="#Решающее-дерево" data-toc-modified-id="Решающее-дерево-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Решающее дерево</a></span></li><li><span><a href="#Случайный-лес" data-toc-modified-id="Случайный-лес-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Случайный лес</a></span></li><li><span><a href="#Промежуточный-вывод" data-toc-modified-id="Промежуточный-вывод-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Промежуточный вывод</a></span></li></ul></li><li><span><a href="#Тестирование" data-toc-modified-id="Тестирование-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Тестирование</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Выводы</a></span></li></ul></div>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

In [1]:
import pandas as pd
import os
import re
import numpy as np
import matplotlib.pyplot as plt
import nltk
from nltk.corpus import stopwords
from nltk.corpus import wordnet
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import WhitespaceTokenizer
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from nltk.corpus import stopwords as nltk_stopwords
nltk.download('stopwords') 
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score
import warnings
warnings.filterwarnings('ignore')
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\bodri\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\bodri\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
path_1 = '/Users/ivanbodrenkov/Yandex.Disk.localized/Яндекс Практикум/13. ML для текстов/Проект/toxic_comments.csv'
path_2 = 'https://code.s3.yandex.net/datasets/toxic_comments.csv'
path_3 = 'C:/Users/bodrenkov_ia/Desktop/Project/toxic_comments.csv'
path_4 = '/datasets/toxic_comments.csv'
path_5 = 'D:/Яндекс Диск/YandexDisk/Яндекс Практикум/13. ML для текстов/Проект/toxic_comments.csv'

if os.path.exists(path_1):
    data = pd.read_csv(path_1, sep = ',')
elif os.path.exists(path_2):
    data = pd.read_csv(path_2, sep = ',')
elif os.path.exists(path_3):
    data = pd.read_csv(path_3, sep = ',')
elif os.path.exists(path_4):
    data = pd.read_csv(path_4, sep = ',')
elif os.path.exists(path_5):
    data = pd.read_csv(path_5, sep = ',')
else:
    print('Что-то пошло не так!')

In [3]:
def series_lemmatizer(text:pd.Series):
    wnl = WordNetLemmatizer()
    wnt = WhitespaceTokenizer()
    
    return np.array([wnl.lemmatize(w) for w in wnt.tokenize(text)])

In [4]:
def text_cleaner(text):
    text = text.lower()
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub('\W', ' ', text)
    text = re.sub('\s+', ' ', text)
    
    text = text.split()
    text = " ".join(text)
    
    return text

In [5]:
def upsample(features, target, repeat):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]
    
    features_upsampled = pd.concat([features_zeros] + [features_ones] * repeat)
    target_upsampled = pd.concat([target_zeros] + [target_ones] * repeat)
    
    features_upsampled, target_upsampled = shuffle(features_upsampled, target_upsampled, random_state=12345)
    
    return features_upsampled, target_upsampled

### Основная информация о данных

In [6]:
data.head()

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0


In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


### Удаление лишних данных

In [8]:
data = data.drop(columns = 'Unnamed: 0', axis = 1)

In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159292 non-null  object
 1   toxic   159292 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


In [10]:
data.head()

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


### Расчет процента токсичных твитов

In [11]:
print('Токсичных твитов во всем датасете', 
      round(100 * data.loc[data['toxic'] == 1].shape[0] / data.loc[data['toxic'] == 0].shape[0], 3), '%')

Токсичных твитов во всем датасете 11.31 %


### Очистка, лемматизация текста

In [12]:
data['text_cleaned'] = data['text'].apply(text_cleaner)

In [13]:
data.head()

,text,toxic,text_cleaned
0,Explanation\nWhy the edits made under my usern...,0,explanation why the edits made under my userna...
1,D'aww! He matches this background colour I'm s...,0,d aww he matches this background colour i am s...
2,"Hey man, I'm really not trying to edit war. It...",0,hey man i am really not trying to edit war it ...
3,"""\nMore\nI can't make any real suggestions on ...",0,more i cannot make any real suggestions on imp...
4,"You, sir, are my hero. Any chance you remember...",0,you sir are my hero any chance you remember wh...


In [14]:
%%time
data['text_lemmatized'] = data['text_cleaned'].apply(series_lemmatizer)

Wall time: 46.3 s


In [15]:
data['text_lemmatized'] = data['text_lemmatized'].apply(lambda text : ' '.join(text))

In [16]:
data.head(20)

,text,toxic,text_cleaned,text_lemmatized
0,Explanation\nWhy the edits made under my usern...,0,explanation why the edits made under my userna...,explanation why the edits made under my userna...
1,D'aww! He matches this background colour I'm s...,0,d aww he matches this background colour i am s...,d aww he match this background colour i am see...
2,"Hey man, I'm really not trying to edit war. It...",0,hey man i am really not trying to edit war it ...,hey man i am really not trying to edit war it ...
3,"""\nMore\nI can't make any real suggestions on ...",0,more i cannot make any real suggestions on imp...,more i cannot make any real suggestion on impr...
4,"You, sir, are my hero. Any chance you remember...",0,you sir are my hero any chance you remember wh...,you sir are my hero any chance you remember wh...
5,"""\n\nCongratulations from me as well, use the ...",0,congratulations from me as well use the tools ...,congratulation from me a well use the tool wel...
6,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1,cocksucker before you piss around on my work,cocksucker before you piss around on my work
7,Your vandalism to the Matt Shirvington article...,0,your vandalism to the matt shirvington article...,your vandalism to the matt shirvington article...
8,Sorry if the word 'nonsense' was offensive to ...,0,sorry if the word nonsense was offensive to yo...,sorry if the word nonsense wa offensive to you...
9,alignment on this subject and which are contra...,0,alignment on this subject and which are contra...,alignment on this subject and which are contra...


In [17]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 4 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   text             159292 non-null  object
 1   toxic            159292 non-null  int64 
 2   text_cleaned     159292 non-null  object
 3   text_lemmatized  159292 non-null  object
dtypes: int64(1), object(3)
memory usage: 4.9+ MB


###  Разделение на выборки

In [18]:
features = data.drop(columns = ['toxic'])
target = data['toxic']

In [19]:
features_train, features_test, target_train, target_test = train_test_split(features, 
                                                                            target, 
                                                                            test_size = 0.25, 
                                                                            random_state = 12345)

In [20]:
print('Размер тестовой выборки:', features_test.shape[0])
print('Размер тренировочной выборки:', features_train.shape[0])

Размер тестовой выборки: 39823
Размер тренировочной выборки: 119469


### Балансировка классов

In [21]:
print('Токсичных твитов в тренировочном датасете', 
      round(100 * target_train.loc[data['toxic'] == 1].shape[0] / target_train.loc[data['toxic'] == 0].shape[0], 3), '%')

Токсичных твитов в тренировочном датасете 11.313 %


In [22]:
features_train_upsampled, target_train_upsampled = upsample(features_train, target_train, 4)

In [23]:
print('Токсичных твитов в тренировочном датасете', 
      round(100 * target_train_upsampled.loc[data['toxic'] == 1].shape[0] / 
            target_train_upsampled.loc[data['toxic'] == 0].shape[0], 3), '%')

Токсичных твитов в тренировочном датасете 45.252 %


### TF-IDF

In [24]:
stop_words = set(nltk_stopwords.words('english')) 

In [25]:
corpus_lemmatized_train = features_train_upsampled['text_lemmatized'].values.astype('U')
print('Размер корпуса:', corpus_lemmatized_train.shape)

Размер корпуса: (155895,)


In [26]:
count_tf_idf = TfidfVectorizer (stop_words = stop_words)
tf_idf_train = count_tf_idf.fit_transform(corpus_lemmatized_train)
print(tf_idf_train.shape)

(155895, 149683)


In [27]:
corpus_lemmatized_test = features_test['text_lemmatized'].values
print('Размер корпуса:', corpus_lemmatized_test.shape)

Размер корпуса: (39823,)


In [28]:
tf_idf_test = count_tf_idf.transform(corpus_lemmatized_test)
print(tf_idf_test.shape)

(39823, 149683)


### Промежуточый вывод

В ходе подготовки данных из датасета был убран столбец unnamed. Для фильтрования разнообразных артефактов в тексте была написана функция text_cleaner, работающая с помощью регулярных выражений. Тексты лемматизированы с помощью NLTK wordnet. Около 11.313 % описаний обучающей выборки состоит из токсичных текстов, поэтому проведена балансировка классов с помощью upsampling. Рассчитаны TF-IDF для обучающей и тестовой выборок.

## Обучение

### Логистическая регрессия

In [29]:
%%time

logistic_regression_grid_space = {'C' : [5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15],
                                  'max_iter' : [50, 100, 200]
                                 }

model_1 = LogisticRegression(random_state = 12345, solver = 'sag', class_weight = 'balanced')

grid = GridSearchCV(estimator = model_1, param_grid = logistic_regression_grid_space, cv = 2, scoring = 'f1')
model_grid = grid.fit(tf_idf_train, target_train_upsampled)

print('Лучшие гиперпараметры:', model_grid.best_params_)
print('Лучшая оценка:', abs(model_grid.best_score_))

Лучшие гиперпараметры: {'C': 15, 'max_iter': 50}
Лучшая оценка: 0.9404879553773938
Wall time: 2min 34s


In [30]:
%%time

model_1 = LogisticRegression(C = int(model_grid.best_params_['C']), 
                             max_iter = int(model_grid.best_params_['max_iter']), 
                             random_state = 12345, 
                             solver = 'sag', 
                             class_weight = 'balanced')
model_1.fit(tf_idf_train, target_train_upsampled)

Wall time: 5.51 s


LogisticRegression(C=15, class_weight='balanced', max_iter=50,
                   random_state=12345, solver='sag')

### Решающее дерево

In [31]:
%%time

decision_tree_grid_space = {'max_depth' : [5, 10, 15],
                            'min_samples_split' : [2, 4, 8],
                            'min_samples_leaf' : [1, 2, 3]
                           }

model_2 = DecisionTreeClassifier(random_state = 12345, class_weight = 'balanced')

grid = GridSearchCV(estimator = model_2, param_grid = decision_tree_grid_space, cv = 2, scoring = 'f1')
model_grid = grid.fit(tf_idf_train, target_train_upsampled)

print('Лучшие гиперпараметры:', model_grid.best_params_)
print('Лучшая оценка:', abs(model_grid.best_score_))

Лучшие гиперпараметры: {'max_depth': 15, 'min_samples_leaf': 1, 'min_samples_split': 4}
Лучшая оценка: 0.6466098088684222
Wall time: 10min 24s


In [32]:
%%time

model_2 = DecisionTreeClassifier(max_depth = int(model_grid.best_params_['max_depth']), 
                                 min_samples_split = int(model_grid.best_params_['min_samples_split']),
                                 min_samples_leaf = int(model_grid.best_params_['min_samples_leaf']),
                                 random_state = 12345, 
                                 class_weight = 'balanced')
model_2.fit(tf_idf_train, target_train_upsampled)

Wall time: 11.3 s


DecisionTreeClassifier(class_weight='balanced', max_depth=15,
                       min_samples_split=4, random_state=12345)

### Случайный лес

In [33]:
%%time

random_forest_grid_space = {'n_estimators' : [10, 100, 200, 400],
                            'max_depth' : [2, 4, 8],
                            'min_samples_leaf' : [1, 2, 3]
                           }

model_3 = RandomForestClassifier(random_state = 12345, class_weight = 'balanced')

grid = GridSearchCV(estimator = model_3, param_grid = random_forest_grid_space, cv = 2, scoring = 'f1')
model_grid = grid.fit(tf_idf_train, target_train_upsampled)

print('Лучшие гиперпараметры:', model_grid.best_params_)
print('Лучшая оценка:', abs(model_grid.best_score_))

Лучшие гиперпараметры: {'max_depth': 8, 'min_samples_leaf': 1, 'n_estimators': 400}
Лучшая оценка: 0.6709660556799433
Wall time: 23min 2s


In [34]:
%%time

model_3 = RandomForestClassifier(n_estimators = int(model_grid.best_params_['n_estimators']), 
                                 max_depth = int(model_grid.best_params_['max_depth']),
                                 min_samples_leaf = int(model_grid.best_params_['min_samples_leaf']),
                                 random_state = 12345, 
                                 class_weight = 'balanced')
model_3.fit(tf_idf_train, target_train_upsampled)

Wall time: 40.4 s


RandomForestClassifier(class_weight='balanced', max_depth=8, n_estimators=400,
                       random_state=12345)

### Промежуточный вывод

Были протестированы три модели:
- логистическая регрессия;
- решающее дерево;
- случайный лес.

Результаты тренировки моделей в таблице ниже:

|  |LogisticRegression|DecisionTreeClassifier|RandomForestClassifier| 
|:-|:-:|:-:|:-:|
|F1|**0.940**|0.647|0.671|
|Время обучения модели|0 мин 5.5 сек|0 мин 11.3 сек|0 мин 40.4 сек|
|Время поиска гиперпараметров|2 мин 34 сек|10 мин 24 сек|23 мин 2 сек|

Наилучшие результаты показала модель логистической регрессии с гиперпараметрами:
- С = 15;
- max_iter = 50;
- solver = 'sag';
- class_weight = 'balanced'.

## Тестирование

In [35]:
%%time
predict_test = model_1.predict(tf_idf_test)

print('Метрики наилучшей модели из исследованных:')                        
print('\nF1 =', round(f1_score(target_test, predict_test), 3))

Метрики наилучшей модели из исследованных:

F1 = 0.765
Wall time: 18.9 ms


## Выводы

В ходе работы над проектом была получена модель, определяющая токсичный или не токсичный текст описания. Тексты были отфильтрованы от разного рода артефактов (например выражений с апострофами), лемматизированы. Трненировочная выборка была сбалансирована с помощью upsampling.

Наилучшей моделью оказалась LogisticRegression с гиперпараметрами:
- С = 15;
- max_iter = 50;
- solver = 'sag';
- class_weight = 'balanced'.

Для наилучшей модели на тестовой выборке F1 = 0.765.